In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 448 kB 40.0 MB/s 


In [2]:
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [3]:
import os

import numpy as np
import pandas as pd

from datetime import datetime
import json
import re

from konlpy.tag import Okt # komoran, han, kkma
from konlpy.tag import Mecab
from konlpy.tag import Komoran

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

from tqdm.notebook import tqdm
import time

In [5]:
import pickle
with open('train.pickle','rb')as fr:
    train = pickle.load(fr)
print(train)

      label                                               제목내용
0         1  한국 전쟁수행능력  일본을 앞섰다 .  정욱식 칼럼  경제력 차이 3배  국방 예산...
1         1   기업을   UP 시켜라  3%에 묶인 대주주 의결권 떼지어 오는 헤지펀드 어떻게 ...
2         1  산업연구원  러 극동서 남 북 러 3각 경협 추진 미리 준비해야 . 한반도 평화 정...
3         1   올림픽 첫 금메달  때   포니... 박항서 기적  때 베트남 첫 완성차. 197...
4         1   50 60 댓글 달지 말고 아세안 가라 는 김현철 경제보좌관은 누구. 현 정부 대...
...     ...                                                ...
2307      0   전국 주요 신문 톱뉴스  24일 조간 .      경향신문   미  러에 본격 제...
2308      0   조간브리핑  고신용자도 대출 금리 4% 넘어서 본격적  고금리 부담  시작. ac...
2309      0  사라지지 않은 러시아 전면 침공 가능성   미 기준금리 전망도 움직이나  글로벌마켓...
2310      0  국제유가 장중 99달러  한국  3%성장  빨간불 .  우크라이나 사태 올 전망치 ...
2311      0   View   Review  커지는 우크라 리스크 원유 수입 5위 한국  문제는 에...

[2312 rows x 2 columns]


In [6]:
import pickle
with open('test.pickle','rb')as fr:
    test = pickle.load(fr)
print(test)

      label                                               제목내용
0         1   정운찬 칼럼 우리가 세워야 할 새로운 나라. 한강을 굽어보는 국립현충원에 봄이 가...
1         1   자본 빨아들이는 북미 셰일가스 좋은 투자 기회 생겨날 것 . 글로벌 에너지PE 퍼...
2         1   중앙아시아 중심 카자흐스탄  기계  인프라  서비스 경제 협력 유망 .    한국...
3         1   김대성의 환율이야기  노르웨이 국부펀드가 한국 채권 배제한 교훈.   자료 한국은...
4         1  베네수엘라 아르헨 그리스 나랏 돈 펑펑 쓰다 경제 파탄. Cover Story   ...
...     ...                                                ...
1752      0  원투펀치 맞은 한국경제   올해 성장률 3% 달성 힘들다 . 대외 의존도 높아 역대...
1753      0  10년새 경험못했던   i 쓰나미 가 닥친다. 우크라 전쟁 상하이항 봉쇄로 원자재 ...
1754      0  승승장구하는 K방산   주가도  꿈틀 . ack           기자 영하 30도...
1755      0  다시 나는 항공편  완전한 회복까지는 먼 길 . ack           앵커 다음...
1756      0  ADB    물가상승률 1.9% 3.2%로 대폭 상향. 우크라사태 글로벌 인플레 영...

[602 rows x 2 columns]


In [7]:
with open('tog.pickle','rb')as fr:
    tog = pickle.load(fr)
print(tog)

      label                                               제목내용
0         1   정운찬 칼럼 우리가 세워야 할 새로운 나라. 한강을 굽어보는 국립현충원에 봄이 가...
1         1   자본 빨아들이는 북미 셰일가스 좋은 투자 기회 생겨날 것 . 글로벌 에너지PE 퍼...
2         1   중앙아시아 중심 카자흐스탄  기계  인프라  서비스 경제 협력 유망 .    한국...
3         1   김대성의 환율이야기  노르웨이 국부펀드가 한국 채권 배제한 교훈.   자료 한국은...
4         1  베네수엘라 아르헨 그리스 나랏 돈 펑펑 쓰다 경제 파탄. Cover Story   ...
...     ...                                                ...
2907      0   전국 주요 신문 톱뉴스  24일 조간 .      경향신문   미  러에 본격 제...
2908      0   조간브리핑  고신용자도 대출 금리 4% 넘어서 본격적  고금리 부담  시작. ac...
2909      0  사라지지 않은 러시아 전면 침공 가능성   미 기준금리 전망도 움직이나  글로벌마켓...
2910      0  국제유가 장중 99달러  한국  3%성장  빨간불 .  우크라이나 사태 올 전망치 ...
2911      0   View   Review  커지는 우크라 리스크 원유 수입 5위 한국  문제는 에...

[2912 rows x 2 columns]


In [9]:
import pickle
with open('stopwords.pickle','rb')as fr:
    stopwords = pickle.load(fr)
print(stopwords)

['그래', '아니', '아', '뭐', '응', '네', '예', '자', '야', '글쎄', '참', '어디', '그럼', '아아', '얘', '임마', '아이고', '여보', '어', '저', '원', '아이구', '음', '글쎄요', '아냐', '어머', '오', '흥', '아이', '에이', '허허', '아니야', '여보세요', '그러나', '그리고', '그런데', '그래서', '따라서', '하지만', '또', '또는', '그러므로', '및', '또한', '그러면', '즉', '그럼', '그러니까', '그렇지만', '오히려', '역시', '그리하여', '다만', '혹은', '그래도', '한편', '이른바', '더구나', '왜냐하면', '근데', '그러자', '더욱이', '하긴', '하기야', '그러면서', '하물며', '그러니', '그러다가', '단', '이리하여', '그', '이', '한', '두', '다른', '그런', '이런', '어떤', '모든', '어느', '몇', '여러', '무슨', '세', '전', '저', '각', '첫', '새', '아무', '약', '네', '아무런', '총', '제', '온', '옛', '오랜', '단', '올', '온갖', '별', '현', '한두', '맨', '양', '몇몇', '수', '딴', '서너', '저런', '두어', '모', '주', '석', '스무', '여느', '이런저런', '본', '동', '웬', '헌', '순', '왠', '요', '두세', '지난', '근', '타', '그까짓', '매', '고', '갖은', '일대', '것', '수', '등', '년', '때문', '일', '중', '월', '씨', '데', '번', '명', '원', '개', '거', '가지', '뿐', '듯', '간', '쪽', '분', '시', '채', '만', '대', '년대', '줄', '놈', '적', '터', '만큼', '바', '측', '내', '편', '차', '자', '세', '대로', '점', '달러', 

In [11]:
X_train = train['제목내용']
y_train = train.label

X_test = test['제목내용']  
y_test = test.label 
X_train[0], y_train[0]

('한국 전쟁수행능력  일본을 앞섰다 .  정욱식 칼럼  경제력 차이 3배  국방 예산 과도하다는 증거  정욱식 평화네트워크 대표  군사력 측정 전문기관인 글로벌 파이어 파워 GFP 가  2018년 잠재적인 전쟁 수행능력 에서 한국이 일본을 앞섰다는 평가를 내놨다. 한국을 136개 평가 대상국 가운데 7위로 평가한 반면 일본은 8위를 기록했다는 것이다. 이 기관이 한국의 전쟁 수행능력을 일본보다 높게 평가한 것은 2006년 이래 처음이다.GFP는 군사력의 양과 질적 측면뿐만 아니라 자원  재정  지리  산업 등 55개 요소로 해당 국가의 전쟁 수행능력을 평가해왔다. 하지만 핵무기 보유고는 중요한 요소에 포함시키지 않았다. 재래식 군사력을 동원한 전쟁 수행 능력을 주된 평가 대상으로 삼았기 때문이다. 이에 따라 비공식 핵보유국들인 이스라엘  파키스탄  북한의 전쟁 수행능력은 각각 16  17  18위로 평가되었다.   글로벌 파이어 파워 가 발표한  2018년 잠재적인 전쟁 수행 능력  순위  www.globalfirepower.com일본을 앞선 핵심 사유는  가용 병력 GFP는 0.0000에 가까울수록 전쟁 능력이 우수한 것이라며  한국은 0.2001로  일본은 0.2107로 점수를 매겼다. 두 나라 앞에는 미국  러시아  중국  인도  프랑스  영국 순으로 순위가 매겨졌다.GFP가 한국을 일본보다 높은 순위에 올려놓은 핵심적인 이유는 정규군과 예비군을 합친  가용 병력 의 차이에 있다. 일본은 약 31만 명인 반면에 한국은 17배에 달하는 약 580만 명에 해당된다는 것이다. 정규군 규모에 있어서도 일본은 약 24만 명인 반면에  한국은 일본보다 2배가 훨씬 넘는 60만 명 정도이다.한편 이 기관은 한국과 일본의 군용기 보유 수는 1560대와 1508대  전차는 2654대와 679대  함정은 166척과 131척이라고 밝혔다. 양적으로는 한국이 많지만 질적 측면에선 일본이 우수하다는 것이 일반적인 평가이기도 하다.한국이 일본을 앞섰다는 GFP의 평가를 얼마나 

In [12]:
# pos_train = tog[tog['num']==1] # 데이터 셔플하는법 # 특정 값을 가지는 변수 창출
# neg_train = tog[tog['num']==0].sample(frac=1)

In [14]:
train.value_counts('label')

label
0    1156
1    1156
dtype: int64

In [15]:
stop_words = stopwords

In [16]:
def preprocessing(news, tokenizer, remove_stopwords = False, stop_words = []):
    news = re.sub("[^가-힣\s]", "", news)
    
    news = tokenizer.nouns(news)

    news = [token for token in news if len(token) > 1]

    if remove_stopwords:
        news = [token for token in news if not token in stop_words]

    return news

In [17]:
tokenizer = Mecab()

In [18]:
x_train = train['제목내용'].apply(lambda x: preprocessing(x, Mecab(), remove_stopwords = True, stop_words=stop_words))

In [19]:
x_test = test['제목내용'].apply(lambda x: preprocessing(x, Mecab(), remove_stopwords = True, stop_words=stop_words))

In [20]:
x_test

0       [정운찬, 칼럼, 나라, 국립, 현충원, 목련, 자태, 홍매화, 핏빛, 호국, 영령...
1       [자본, 북미, 셰일가스, 투자, 기회, 글로벌, 에너지, 퍼스트, 리저브, 알렉스...
2       [중앙아시아, 중심, 카자흐스탄, 기계, 인프라, 서비스, 경제, 협력, 유망, 한...
3       [김대성, 환율, 이야기, 노르웨이, 국부, 펀드, 한국, 채권, 배제, 교훈, 자...
4       [베네수엘라, 아르, 그리스, 경제, 파탄, 국가, 부채, 사상, 최대, 선심, 복...
                              ...                        
1752    [원투, 펀치, 한국, 경제, 올해, 성장률, 달성, 대외, 의존, 역대, 충격, ...
1753    [경험, 나미, 우크라, 전쟁, 상하, 이항, 봉쇄, 원자재, 식량, 수입, 물가,...
1754    [승승장구, 방산, 주가, 기자, 영하, 노르웨이, 설원, 국산, 전차, 안정, 방...
1755    [항공편, 완전, 회복, 앵커, 다음, 코로나, 국제선, 하늘길, 본격, 항공업, ...
1756    [물가, 상승, 상향, 우크라, 사태, 글로벌, 인플레, 영향, 긴축, 확대, 발표...
Name: 제목내용, Length: 602, dtype: object

In [21]:
x_train

0       [한국, 전쟁, 수행, 능력, 일본, 정욱식, 칼럼, 경제력, 차이, 국방, 예산,...
1       [기업, 대주주, 의결권, 헤지, 펀드, 기업지배구조, 소버린, 엘리엇, 사태, 무...
2       [산업, 연구원, 극동, 경협, 추진, 준비, 한반도, 평화, 정착, 과정, 협력,...
3       [올림픽, 금메달, 포니, 박항서, 기적, 베트남, 완성, 양정모, 레슬링, 금메달...
4       [댓글, 아세안, 김현철, 경제, 보좌관, 정부, 대외, 경제, 정책, 밑그림, 핵...
                              ...                        
2307    [전국, 주요, 신문, 톱뉴스, 조간, 경향신문, 본격, 제재, 대화, 국민, 일보...
2308    [조간, 브리핑, 고신, 용자, 대출, 금리, 본격, 고금리, 부담, 시작, 경제,...
2309    [러시아, 전면, 침공, 가능, 기준, 금리, 전망, 글로벌, 마켓, 앵커, 미국,...
2310    [국제, 유가, 장중, 한국, 성장, 빨간불, 우크라, 사태, 전망, 전제, 러시아...
2311    [우크라, 리스크, 원유, 수입, 한국, 문제, 에너지, 러시아, 우크라, 침공, ...
Name: 제목내용, Length: 2312, dtype: object

In [22]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train) 
x_train_seq = tokenizer.texts_to_sequences(x_train) 

x_test_seq = tokenizer.texts_to_sequences(x_test)

word_vocab = tokenizer.word_index 
print("전체 단어 개수: ", len(word_vocab))

전체 단어 개수:  26687


In [23]:
x_len = x_train.map(lambda x: len(x))
x_len.sort_values(ascending=False)[:round(len(x_len)*0.1)]

889     3683
1235    2650
79      2595
1901    2571
857     2532
        ... 
140      682
1278     681
100      679
2216     679
670      677
Name: 제목내용, Length: 231, dtype: int64

In [24]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

model = Sequential()
model.add(Embedding(len(word_vocab)+1, 30)) 
model.add(LSTM(16))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 30)          800640    
                                                                 
 lstm (LSTM)                 (None, 16)                3008      
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 803,665
Trainable params: 803,665
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
MAX_SEQUENCE_LENGTH = 5 # 문장 최대 길이

x_train_pad = pad_sequences(x_train_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='post') # 문장의 길이가 30 단어가 넘어가면 자르고, 모자르면 0으로 채워 넣는다.

print('Shape of input data tensor:', x_train_pad.shape) # 리뷰 데이터의 형태 확인
print('Shape of label tensor:', y_train.shape) # 감정 데이터 형태 확인

x_test_pad = pad_sequences(x_test_seq, maxlen=MAX_SEQUENCE_LENGTH) # 문장의 길이가 30 단어가 넘어가면 자르고, 모자르면 0으로 채워 넣는다.

print('Shape of input data tensor:', x_test_pad.shape) # 리뷰 데이터의 형태 확인
print('Shape of label tensor:', y_test.shape) # 감정 데이터 형태 확인

Shape of input data tensor: (2312, 5)
Shape of label tensor: (2312,)
Shape of input data tensor: (602, 5)
Shape of label tensor: (602,)


In [26]:
x_train_pad

array([[ 1558,   124,   132,   118,    78],
       [   99,   124,   132,   118,    78],
       [  822,   124,   132,   118,    78],
       ...,
       [  206,   277,     2,     1,  4050],
       [ 1183,    20,  1502,  4033, 10038],
       [   29,   604,    82,   380,   763]], dtype=int32)

In [27]:
x_train_pad.shape

(2312, 5)

In [28]:
model.fit(x_train_pad, y_train, epochs=5, batch_size=16, validation_split=0.2)

Epoch 1/5
116/116 [==============================] - 5s 16ms/step - loss: 0.4029 - accuracy: 0.7896 - val_loss: 0.3999 - val_accuracy: 0.9849
Epoch 2/5
116/116 [==============================] - 1s 10ms/step - loss: 0.1152 - accuracy: 0.9740 - val_loss: 0.1028 - val_accuracy: 0.9914
Epoch 3/5
116/116 [==============================] - 1s 10ms/step - loss: 0.0730 - accuracy: 0.9762 - val_loss: 0.0794 - val_accuracy: 0.9870
Epoch 4/5
116/116 [==============================] - 1s 10ms/step - loss: 0.0559 - accuracy: 0.9838 - val_loss: 0.0816 - val_accuracy: 0.9784
Epoch 5/5
116/116 [==============================] - 1s 10ms/step - loss: 0.0455 - accuracy: 0.9870 - val_loss: 0.0874 - val_accuracy: 0.9719


In [29]:
model.fit(x_train_pad, y_train, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
58/58 [==============================] - 1s 12ms/step - loss: 0.0365 - accuracy: 0.9897 - val_loss: 0.0882 - val_accuracy: 0.9719
Epoch 2/5
58/58 [==============================] - 1s 11ms/step - loss: 0.0302 - accuracy: 0.9930 - val_loss: 0.0848 - val_accuracy: 0.9719
Epoch 3/5
58/58 [==============================] - 1s 11ms/step - loss: 0.0250 - accuracy: 0.9935 - val_loss: 0.0892 - val_accuracy: 0.9719
Epoch 4/5
58/58 [==============================] - 1s 11ms/step - loss: 0.0200 - accuracy: 0.9941 - val_loss: 0.0922 - val_accuracy: 0.9698
Epoch 5/5
58/58 [==============================] - 1s 11ms/step - loss: 0.0153 - accuracy: 0.9941 - val_loss: 0.0889 - val_accuracy: 0.9698


In [30]:
model1 = model.fit(x_train_pad, y_train, epochs=5, batch_size=16, validation_split=0.2)

Epoch 1/5
116/116 [==============================] - 1s 10ms/step - loss: 0.0118 - accuracy: 0.9962 - val_loss: 0.0955 - val_accuracy: 0.9654
Epoch 2/5
116/116 [==============================] - 1s 10ms/step - loss: 0.0099 - accuracy: 0.9968 - val_loss: 0.1010 - val_accuracy: 0.9611
Epoch 3/5
116/116 [==============================] - 1s 10ms/step - loss: 0.0072 - accuracy: 0.9978 - val_loss: 0.0999 - val_accuracy: 0.9611
Epoch 4/5
116/116 [==============================] - 1s 10ms/step - loss: 0.0059 - accuracy: 0.9984 - val_loss: 0.1116 - val_accuracy: 0.9611
Epoch 5/5
116/116 [==============================] - 1s 10ms/step - loss: 0.0046 - accuracy: 0.9989 - val_loss: 0.1287 - val_accuracy: 0.9525


In [31]:
model.fit(x_train_pad, y_train, epochs=5, batch_size=8, validation_split=0.2)

Epoch 1/5
232/232 [==============================] - 2s 10ms/step - loss: 0.0040 - accuracy: 0.9989 - val_loss: 0.1674 - val_accuracy: 0.9525
Epoch 2/5
232/232 [==============================] - 2s 10ms/step - loss: 0.0056 - accuracy: 0.9984 - val_loss: 0.1946 - val_accuracy: 0.9503
Epoch 3/5
232/232 [==============================] - 2s 10ms/step - loss: 0.0043 - accuracy: 0.9989 - val_loss: 0.1904 - val_accuracy: 0.9503
Epoch 4/5
232/232 [==============================] - 2s 10ms/step - loss: 0.0034 - accuracy: 0.9995 - val_loss: 0.2137 - val_accuracy: 0.9503
Epoch 5/5
232/232 [==============================] - 2s 10ms/step - loss: 0.0033 - accuracy: 0.9995 - val_loss: 0.2627 - val_accuracy: 0.9438


In [32]:
# model.fit(x_train_pad, y_train, epochs=5, batch_size=4, validation_split=0.2)

In [37]:
model.fit(x_train_pad, y_train, epochs=5, batch_size=32, validation_split=0.1)

Epoch 1/5
65/65 [==============================] - 1s 10ms/step - loss: 7.0903e-04 - accuracy: 0.9995 - val_loss: 0.1840 - val_accuracy: 0.9569
Epoch 2/5
65/65 [==============================] - 1s 10ms/step - loss: 2.8330e-04 - accuracy: 1.0000 - val_loss: 0.1874 - val_accuracy: 0.9569
Epoch 3/5
65/65 [==============================] - 1s 11ms/step - loss: 1.0735e-04 - accuracy: 1.0000 - val_loss: 0.2027 - val_accuracy: 0.9569
Epoch 4/5
65/65 [==============================] - 1s 10ms/step - loss: 5.2502e-05 - accuracy: 1.0000 - val_loss: 0.2108 - val_accuracy: 0.9569
Epoch 5/5
65/65 [==============================] - 1s 10ms/step - loss: 2.2573e-05 - accuracy: 1.0000 - val_loss: 0.2245 - val_accuracy: 0.9569


In [34]:
model.evaluate(x_test_pad, y_test)

19/19 [==============================] - 0s 3ms/step - loss: 0.2466 - accuracy: 0.9734


[0.24656030535697937, 0.9734219312667847]

In [35]:
model.evaluate(x_test_pad, y_test)

19/19 [==============================] - 0s 2ms/step - loss: 0.2466 - accuracy: 0.9734


[0.24656030535697937, 0.9734219312667847]

In [36]:
# 모델 저장 to HDF5 file
model.save('noun_model.h5')